In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from skimage.feature import local_binary_pattern
from scipy.fft import fft

# Load the HSV reference image or create a default one if file not found
hsv_reference = cv2.imread("hsv-0.png")
if hsv_reference is None:
    # Create a default HSV reference image
    hsv_reference = np.zeros((100, 300, 3), dtype=np.uint8)
    # Add some color information for visualization
    cv2.putText(hsv_reference, "HSV Reference", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
def fire_color_mask(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # More restrictive HSV range for fire colors
    lower_bound1 = np.array([0, 150, 150], dtype=np.uint8)   # Red hues
    upper_bound1 = np.array([10, 255, 255], dtype=np.uint8)
    lower_bound2 = np.array([170, 150, 150], dtype=np.uint8) # Red hues (wrap-around)
    upper_bound2 = np.array([179, 255, 255], dtype=np.uint8)
    lower_bound3 = np.array([20, 150, 150], dtype=np.uint8) # Orange-yellow hues
    upper_bound3 = np.array([30, 255, 255], dtype=np.uint8)
    lower_bound2 = np.array([160, 50, 50], dtype=np.uint8) # Red hues (wrap-around)
    upper_bound2 = np.array([179, 255, 255], dtype=np.uint8)
    lower_bound3 = np.array([18, 100, 100], dtype=np.uint8) # Orange-yellow hues
    upper_bound3 = np.array([35, 255, 255], dtype=np.uint8)
    
    # Combine masks for red and orange-yellow hues
    mask1 = cv2.inRange(hsv, lower_bound1, upper_bound1)
    mask2 = cv2.inRange(hsv, lower_bound2, upper_bound2)
    mask3 = cv2.inRange(hsv, lower_bound3, upper_bound3)
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.bitwise_or(mask, mask3)
    
    # Morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and filter based on size and shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Increased area threshold for better accuracy
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 2.0:  # Filter based on aspect ratio to avoid false positives
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Fire Detected!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return frame, mask

def detect_motion(frame, background_subtractor):
    fg_mask = background_subtractor.apply(frame)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    return fg_mask

from scipy.fft import fft

def check_flicker(intensity_history, sample_rate=30):
    if len(intensity_history) < sample_rate:
        return False
    yf = fft(intensity_history[-sample_rate:])
    xf = np.fft.fftfreq(sample_rate, 1/sample_rate)
    dominant_freq = np.abs(xf[np.argmax(np.abs(yf))])
    return 8 < dominant_freq < 12  # Check for ~10 Hz flicker

from skimage.feature import local_binary_pattern

def analyze_texture(roi_gray):
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(roi_gray, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), density=True)
def process_frame(frame, background_subtractor, intensity_history):
    # Fire color detection
    _, fire_mask = fire_color_mask(frame)  # Unpack the tuple correctly
    
    # Motion detection
    motion_mask = detect_motion(frame, background_subtractor)
    
    # Combine fire mask and motion mask
    combined_mask = cv2.bitwise_and(fire_mask, motion_mask)
    
    # Get ROI for further analysis
    fire_roi = cv2.bitwise_and(frame, frame, mask=combined_mask)
    
    # Flicker analysis
    gray = cv2.cvtColor(fire_roi, cv2.COLOR_BGR2GRAY)
    intensity = np.mean(gray) if np.any(gray) else 0
    intensity_history.append(intensity)
    is_flickering = check_flicker(intensity_history)
    
    # Draw detection results
    result_frame = frame.copy()
    if is_flickering and np.sum(combined_mask) > 5000:
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 1500:  # Increased threshold
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(result_frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(result_frame, "Fire!", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    return result_frame, combined_mask

# Main loop
cap = cv2.VideoCapture(0)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=50, varThreshold=16)
intensity_history = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    # Resize HSV reference image if it exists
    if hsv_reference is not None:
        hsv_ref_resized = cv2.resize(hsv_reference, (300, 100))
    else:
        hsv_ref_resized = np.zeros((100, 300, 3), dtype=np.uint8)
    if not ret:
        break
    processed_frame, mask = fire_color_mask(frame)  # Using the correct function name
    
    # Resize HSV reference image
    hsv_ref_resized = cv2.resize(hsv_reference, (300, 100))
    
    # Display results
    cv2.imshow("Fire Detection", processed_frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("HSV Reference", hsv_ref_resized)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
# Initialize
background_subtractor = cv2.createBackgroundSubtractorMOG2()
intensity_history = []

while True:
    ret, frame = cap.read()
    if not ret: break

    # Step 1: Color Masking
    fire_roi = fire_color_mask(frame)

    # Step 2: Motion Detection
    motion_mask = detect_motion(frame, background_subtractor)

    # Step 3: Flicker Analysis
    gray = cv2.cvtColor(fire_roi, cv2.COLOR_BGR2GRAY)
    intensity = np.mean(gray) if np.any(gray) else 0
    intensity_history.append(intensity)
    is_flickering = check_flicker(intensity_history)

    # Step 4: Texture Analysis
    texture_feature = analyze_texture(gray)

    # Combine Features for Final Decision
    if (np.sum(motion_mask) > 1000 and is_flickering and np.sum(fire_roi) > 5000):
        print("Fire detected!")

    
    def forward(self, x):
        return torch.sigmoid(self.model(x))

def custom_loss(y_pred, y_true, physics_loss_weight=0.1):
    bce_loss = nn.BCELoss()(y_pred, y_true)
    
    # Physics loss: Penalize predictions violating fire flicker/color laws
    physics_loss = torch.mean(torch.relu(8 - y_pred_freq) + torch.relu(y_pred_freq - 12))
    
    return bce_loss + physics_loss_weight * physics_loss

model = FireDetector().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = custom_loss

for epoch in range(10):
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x.cuda())
        loss = criterion(outputs, batch_y.cuda(), physics_loss_weight=0.1)
        loss.backward()
        optimizer.step()
        
def predict_fire(frame_sequence):
    # Preprocess frames (resize, normalize)
    frames = [transform(frame) for frame in frame_sequence]
    frames = torch.stack(frames).unsqueeze(0).cuda()  # Add batch dim
    
    with torch.no_grad():
        prob = model(frames).item()
    
    return prob > 0.5  # Fire if probability > 50%




error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 

In [3]:
%pip install torch-cam

import cv2
import numpy as np
from scipy.fft import fft
from skimage.feature import local_binary_pattern
import torch
import torch.nn as nn
from torchvision import transforms, models
from torchcam.methods import GradCAM  # For explainability

# ----------------------
# 1. INITIALIZATION
# ----------------------
cap = cv2.VideoCapture(0)
background_subtractor = cv2.createBackgroundSubtractorMOG2()
intensity_history = []

# Load HSV reference image
hsv_reference = cv2.imread("hsv-0.png") or np.zeros((100, 300, 3), dtype=np.uint8)

# ----------------------
# 2. FIRE DETECTION FUNCTIONS
# ----------------------
def fire_color_mask(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Adjusted HSV range for fire-like colors (red, orange, yellow)
    lower_bound1 = np.array([0, 50, 50], dtype=np.uint8)   # Red hues
    upper_bound1 = np.array([10, 255, 255], dtype=np.uint8)
    lower_bound2 = np.array([160, 50, 50], dtype=np.uint8) # Red hues (wrap-around)
    upper_bound2 = np.array([179, 255, 255], dtype=np.uint8)
    lower_bound3 = np.array([18, 100, 100], dtype=np.uint8) # Orange-yellow hues
    upper_bound3 = np.array([35, 255, 255], dtype=np.uint8)
    
    # Combine masks for red and orange-yellow hues
    mask1 = cv2.inRange(hsv, lower_bound1, upper_bound1)
    mask2 = cv2.inRange(hsv, lower_bound2, upper_bound2)
    mask3 = cv2.inRange(hsv, lower_bound3, upper_bound3)
    mask = cv2.bitwise_or(mask1, mask2)
    mask = cv2.bitwise_or(mask, mask3)
    
    # Morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and filter based on size and shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Increased area threshold for better accuracy
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if 0.5 < aspect_ratio < 2.0:  # Filter based on aspect ratio to avoid false positives
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Fire Detected!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return frame, mask

    mask = cv2.bitwise_or(
        cv2.inRange(hsv, lower1, upper1),
        cv2.inRange(hsv, lower2, upper2)
    )
    mask = cv2.bitwise_or(mask, cv2.inRange(hsv, lower3, upper3))
    
    kernel = np.ones((5,5), np.uint8)
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

def detect_motion(frame):
    fg_mask = background_subtractor.apply(frame)
    return cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))

def check_flicker(history, sample_rate=30):
    if len(history) < sample_rate: return False
    yf = fft(history[-sample_rate:])
    dominant_freq = np.abs(np.fft.fftfreq(sample_rate, 1/sample_rate)[np.argmax(np.abs(yf))])
    return 8 < dominant_freq < 12  # Fire flickers at ~10Hz

def analyze_texture(roi_gray):
    lbp = local_binary_pattern(roi_gray, 24, 3, method='uniform')
    return np.histogram(lbp, bins=np.arange(27), density=True)[0]

# ----------------------
# 3. DEEP LEARNING MODEL
# ----------------------
class FireDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()
        self.lstm = nn.LSTM(512, 128, batch_first=True)
        self.classifier = nn.Linear(128, 1)

    def forward(self, x):
        batch_size, seq_len = x.shape[0], x.shape[1]
        features = self.cnn(x.view(-1, *x.shape[2:]))
        _, (hidden, _) = self.lstm(features.view(batch_size, seq_len, -1))
        return torch.sigmoid(self.classifier(hidden[-1]))

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FireDetector().to(device)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ----------------------
# 4. MAIN PROCESSING LOOP
# ----------------------
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret: break
    
    # Traditional Computer Vision
    mask = fire_color_mask(frame)
    motion_mask = detect_motion(frame)
    gray = cv2.cvtColor(cv2.bitwise_and(frame, frame, mask=mask), cv2.COLOR_BGR2GRAY)
    
    # Feature Extraction
    intensity_history.append(np.mean(gray) if np.any(gray) else 0)
    is_flickering = check_flicker(intensity_history)
    texture_feat = analyze_texture(gray)
    
    # Deep Learning
    frame_buffer.append(transform(frame))
    if len(frame_buffer) == 30:  # Process every 1 sec (30fps)
        with torch.no_grad():
            prob = model(torch.stack(frame_buffer).unsqueeze(0).to(device)).item()
        frame_buffer = []
    
    # Decision Fusion
    cv_decision = (np.sum(motion_mask) > 1000 and is_flickering and np.sum(mask) > 5000)
    dl_decision = prob > 0.7 if len(frame_buffer)==0 else False
    
    if cv_decision or dl_decision:
        x,y,w,h = cv2.boundingRect(cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0][0])
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
        cv2.putText(frame, "FIRE DETECTED", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    # Display
    cv2.imshow("Live", frame)
    cv2.imshow("Fire Mask", mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch-cam (from versions: none)
ERROR: No matching distribution found for torch-cam


ModuleNotFoundError: No module named 'torchcam'